In [0]:
import requests
import time
import json
import http.client
import datetime as dt
import base64
import uuid

from pyspark.dbutils import DBUtils

class ApiConnector:

    def __init__(self, scope, base_url, headers=None):
        self.base_url = dbutils.secrets.get(scope=scope, key=base_url)
        self.headers = headers or {}


    def request(self, method, endpoint, params=None, data=None, json=None, retry_delay=0.3, max_retries=5, timeout=30):
        retry_count = 0
        while retry_count < max_retries:
            
            if 'http' in self.base_url:
                url = f'{self.base_url}/{endpoint}'
            else:
                url = f'https://{self.base_url}/{endpoint}'

            response = requests.request(
                method=method,
                url=url,
                headers=self.headers,
                params=params,
                data=data,
                json=json,
                timeout=timeout
            )

            if response.status_code in [200, 202]:
                return response

            else:
                retry_count += 1
                if retry_count < max_retries:
                    time.sleep(retry_delay*2)
        
        return response  